# GOFevaluation Exercises
[Robert Hammann](mailto:robert.hammann@outlook.com)  
January 2023

The best way to start with the `GOFevaluation` package is probably to have a look at the [readme](https://github.com/XENONnT/GOFevaluation) on GitHub. It lists all available GOF tests and the corresponding classes.
You can also find some examples in the readme as well as in the example notebook.
But of course, it is much more fun to explore the functionalities yourself!

In the following exercises, we will go through the most important functions of the package and learn how to properly use them.

In [ ]:
import GOFevaluation as ge
import scipy.stats as sps
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from iminuit import Minuit

In [ ]:
mpl.rcParams["figure.dpi"] = 200
mpl.rcParams["figure.figsize"] = [3.5, 2.72]
mpl.rcParams["font.family"] = "serif"
mpl.rcParams["font.size"] = 9

## Generate toy data

In [ ]:
n_data_sample = 300

# reference 0 is drawn from the same distribution as the data ('good fit'),
# 1 is shifted wrt the true distribution ('bad fit')
data_model = sps.multivariate_normal(mean=[0, 0], cov=[[2, 1], [1, 1]])
ref_model_0 = data_model
ref_model_1 = sps.multivariate_normal(mean=[0, 0.7], cov=[[2, 1], [1, 1]])

# Draw samples:
data = pd.DataFrame(data_model.rvs(n_data_sample, random_state=40), columns=["x", "y"])

ref_0 = pd.DataFrame(ref_model_0.rvs(n_data_sample * 100, random_state=41), columns=["x", "y"])
ref_1 = pd.DataFrame(ref_model_1.rvs(n_data_sample * 100, random_state=42), columns=["x", "y"])

In [ ]:
# Visualize the data and the two 'fits'
# Only plot the first 1000 points from the reference samples for readibility
fig, ax = plt.subplots(figsize=(3, 3))

ax.scatter(
    ref_0.x[:1000], ref_0.y[:1000], s=2, c="dodgerblue", alpha=1, label='Ref. Sample 0 ("good fit")'
)
ax.scatter(
    ref_1.x[:1000], ref_1.y[:1000], s=2, c="crimson", alpha=1, label='Ref. Sample 1 ("bad fit")'
)
ax.scatter(data.x, data.y, s=4, c="k", label="Data")

# Cosmetics
ax.legend(frameon=False, loc="upper left")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_ylim(-3, 6.5)

## Initialization
Each GOF test is implemented as a class. As pointed out above, you can find an overview in the readme. In the following, we call an instance of such a class `gof_object`.
Depending on whether you want to perform a binned or unbinned test and whether your data and references are in binned or unbinned format, there are several ways to initialize a `gof_object`, some of which will be used in the following.
You can look up the different initialization methods by looking into the docstring of the class, i.e. <kbd>⇧ Shift</kbd> + <kbd>⇥ Tab</kbd> after the class.
A typical initialization could be `gof_object = ge.KSTestTwoSampleGOF(data_sample, reference_sample)` or `gof_object = ge.BinnedPoissonChi2GOF.bin_equiprobable(...)`. 

<b>Tip: </b> Unfortunately, the docstrings of the individual init methods are not (yet) too informative. Check the docstring of the class to get more information on the arguments!

## Performing a GOF test
All inits create an instance of a class corresponding to the respective GOF test.
The method `.get_pvalue()` can then be used to calculate the test statistic and p-value of the test.
If you are only interested in the value of the test statistic, you can use the `.get_gof()` method.

Depending on whether you perform a binned or an unbinned test, the `.get_pvalue()` method takes the number of Poisson sampled toys (`n_mc`) or the number of permutations (`n_perm`) as an input.
If you need a more precise result for your p-value (e.g. if it is close to the decision boundary), you can rerun the method with a larger corresponding number.

<div style="    
background-color: #BCD7FA;
color: #1E3248;
border-style: solid;
border-color: #1E3248;
border-radius: 4pt; 
padding-left: 20pt;
padding-bottom: 15pt;
" markdown="1"> 
  
<h1>Exercise 1</h1>
Perform a one-dimensional Anderson-Darling test in $x$ for both reference models (<code>ref_0</code> and <code>ref_1</code>). Repeat the tests in $y$. How do you interpret the four p-values?
    
<details markdown="1">
<summary markdown="1">
    <b>Hint</b> on the initialization:
</summary>
    In the docstring of <code>ge.ADTestTwoSampleGOF</code> you can find that the only two inputs are one-dimensional arrays of unbinned data (e.g. <code>data.x</code>) and a corresponding unbinned reference sample (e.g. <code>ref_1.x</code>). The returned object can then be used to calculate the p-value of the GOF test as explained above.
</details>
</div>

## Equiprobably binned tests

<div style="    
background-color: #BCD7FA;
color: #1E3248;
border-style: solid;
border-color: #1E3248;
border-radius: 4pt; 
padding-left: 20pt;
padding-bottom: 15pt;
" markdown="1"> 
  
<h1>Exercise 2</h1>
    a)   Perform a two-dimensional equiprobably binned $\chi^2$-test for both reference models.<br>
    b)   Make a nice plot of the equiprobably binned data and overlay the data points to verify what's going on. <i>(Tip: There is one easy way and a few more cumbersome ways to do this.)</i><br>
    c)   Perform the same tests but change the order in which the partitioning is performed (i.e. first partition in $y$, then partition in $x$)<br><br>


<details markdown="1">
<summary markdown="1">
    <b>Hint</b> on the number of bins:
</summary>
The formula for the "optimal number of bins" is $k_\mathrm{opt}\simeq 3.12 (N-1)^{2/5}$, where $N$ is the number of data points. <code>np.prod(n_partitions)</code> should be close to this value.
</details>

    
<details markdown="1">
<summary markdown="1">
    <b>Hint</b> on the initialization:
</summary>
In the docstring of <code>ge.BinnedChi2GOF</code> or <code>ge.BinnedPoissonChi2GOF</code> you can find that to get equiprobably binned data, you can use <code>.bin_equiprobable(...)</code> i.e. <code>ge.BinnedChi2GOF.bin_equiprobable(data_sample, reference_sample, nevents_expected, n_partitions, ...)</code>. 
</details>


<details markdown="1">
<summary markdown="1">
    <b>Hint</b> on the number of expected events:
</summary>
This one can be tricky. If the integral under your fit function (i.e. the normalization to scale the pdf) is a free parameter of your fit (e.g. extended ML fit - see exercise 3), you should use this value. Otherwise, it is the number of data points.
</details>


<details markdown="1">
<summary markdown="1">
    <b>Hint</b> on the correct input format:
</summary>
Unfortunately, right now the package is very picky when it comes to input formatting. As you might have noticed, it complains when you feed it with the dataframe. The required format is an nD array of shape (<code>sample_size</code>, <code>n_dim</code>). In this case, you can try <code>np.asarray(data)</code>, which transforms the dataframe into an array of shape (300, 2).
</details>

</div>

<div style="    
background-color: #BCD7FA;
color: #1E3248;
border-style: solid;
border-color: #1E3248;
border-radius: 4pt; 
padding-left: 20pt;
padding-bottom: 15pt;
" markdown="1"> 
  
<h1>Exercise 3</h1>
    a)  Perform an extended unbinned maximum likelihood fit of the toy data with a bivariate Gaussian (<code>two_dimensional_normal</code> defined in the next cell).<br>
    b)  Generate a reference sample from the fit pdf.<br>
    c)  Perform a 2d equiprobably binned $\chi^2$-test. Make sure to put the correct value for <code>nevents_expected</code>!<br><br>
    
    
    
<details markdown="1">
<summary markdown="1">
    <b>Hint</b> on the extended likelihood:
</summary>
    In an extended ML fit, the total expectation value becomes a fit parameter. For this, the usual likelihood function is multiplied with the Poisson probability to find <code>len(data)</code> $= n$ events with mean $\nu$, which is now a parameter of the fit. This can be simplified to obtain $-\ln L = \nu(\mathbf{\theta}) - \sum_{i=1}^n \ln(\nu(\mathbf{\theta}) f(x_i, y_i| \mathbf{\theta}))$.
</details>

<details markdown="1">
<summary markdown="1">
    <b>Code snippet</b> of the likelihood function:
</summary>
<pre><code>def extendedNegativeLogLikelihood(mu_x, mu_y, sigma_x, sigma_y, rho, nu):
    '''Returns the extended negative log likelihood'''
    enll = nu - np.sum(np.log(nu * two_dimensional_normal(x, y, mu_x, mu_y, sigma_x, sigma_y, rho)))
    return enll
</code></pre>
</details>

<details markdown="1">
<summary markdown="1">
    <b>Code snippet</b> of the fit:
</summary>
<pre><code># put some reasonable starting points for the fit, then 
# miminize the extended nLL
m = Minuit(extendedNegativeLogLikelihood, 
           mu_x=np.mean(x),
           mu_y=np.mean(y),
           sigma_x=np.std(x),
           sigma_y=np.std(y),
           rho=.9,
           nu=len(data)
          )
m.errordef = Minuit.LIKELIHOOD
m.migrad()
</code></pre>
</details>

<details markdown="1">
<summary markdown="1">
    <b>Code snippet</b> of the sampling:
</summary>
<pre><code># Sample via a simple Metropolis-Hastings algorithm
# (there are for sure much nicer methods and implementations):

n_burnin = 100  # burn-in steps will be discarded in the end
n_samples = int(300*len(data)) + n_burnin
ref_sample = np.zeros((n_samples, 2))
step = np.random.rand(n_samples, 2)*2 - 1  # proposed 2D-step
r = np.random.rand(n_samples)  # random value between 0 and 1

for i in range(n_samples-1):
    ratio = (two_dimensional_normal(ref_sample[i][0] + step[i][0],
                                    ref_sample[i][1] + step[i][1],
                                    *m.values[:-1])
             /two_dimensional_normal(ref_sample[i][0],
                                     ref_sample[i][1],
                                     *m.values[:-1]))
    if ratio > r[i]:
        ref_sample[i+1] = ref_sample[i] + step[i]
    else:
        ref_sample[i+1] = ref_sample[i]

ref_sample = ref_sample[n_burnin:]  # discard burn-in phase
</code></pre>
</details>
</div>

In [ ]:
x = data.x
y = data.y


def two_dimensional_normal(x, y, mu_x, mu_y, sigma_x, sigma_y, rho):
    """
    PDF of a 2D Gauss distribution for coordinates x and y with means
    mu_x and mu_y, widths sigma_x and sigma_y, and correlation rho between
    x and y.
    """
    norm = 1 / (2 * np.pi * sigma_x * sigma_y * np.sqrt(1 - rho**2))
    return norm * np.exp(
        -1
        / (2 * (1 - rho**2))
        * (
            ((x - mu_x) / sigma_x) ** 2
            - 2 * rho * ((x - mu_x) / sigma_x) * ((y - mu_y) / sigma_y)
            + ((y - mu_y) / sigma_y) ** 2
        )
    )